In [6]:
from __future__ import unicode_literals
import json
import glob

import pickle
import os
import re
import pandas as pd
from pprint import pprint

import glob
# from lxml import html


from bs4 import BeautifulSoup

files = glob.glob('scotus/*json')

            
attributes_list = ['absolute_url', 'author', 
                   'author_str', 'cluster', 
                   'date_created','date_modified', 
                   'download_url', 'extracted_by_ocr',
                   'html','html_columbia', 
                   'html_lawbox','html_with_citations', 
                   'id','joined_by', 'local_path', 
                   'opinions_cited','page_count','per_curiam','plain_text','resource_uri', 'sha1', 'type']


attributes = {'absolute_url': 'unknown','author': 'unknown','author_str': 'unknown','cluster': 'unknown','date_created': 'unknown','date_modified': 'unknown','download_url': 'unknown','extracted_by_ocr': 'unknown','html': 'unknown','html_columbia': 'unknown','html_lawbox': 'unknown','html_with_citations': 'unknown','id': 'unknown','joined_by': 'unknown','local_path': 'unknown','opinions_cited': 'unknown','page_count': 'unknown','per_curiam': 'unknown','plain_text': 'unknown','resource_uri': 'unknown','sha1': 'unknown','type': 'unknown'}

In [7]:
class EmptyDict(dict):
    def __missing__(self, key):
        return ''

In [8]:
cases = []
for file in files:
    with open(file, 'r') as f:
        data = json.loads(f.read(), object_hook=EmptyDict)
        cases.append(data)

KeyboardInterrupt: 

In [ ]:
cases

In [ ]:
df = pd.DataFrame(cases)
# df.head()

In [ ]:
df.columns

In [ ]:
df.page_count.unique()

In [ ]:
df.per_curiam.value_counts()

In [ ]:
df.type.value_counts()

In [ ]:
df.joined_by.value_counts()

In [ ]:
df.plain_text = df.plain_text.apply(lambda x: x.replace('\n', ' '))

# Regex Omnibus Section

### Get some missing data (case names, dates)

In [ ]:
date_of_decision = re.search(r"Decided ([A-Z][a-z]+ [0-9]+, [0-9]{4})", df.html_with_citations.iloc[0]).group(0)
print(date_of_decision)

In [ ]:
date_of_argument = re.search(r"Argued ([A-Z][a-z]+ [0-9]+, [0-9]{4})", df.html_with_citations.iloc[0]).group(0)
print(date_of_argument)

In [ ]:
short_name = re.search(r"((?<!\\opinion\\\d)([\w-]+v(.)?[\w-]+)(?!\\))", df.local_path.iloc[0]).group(0).replace('_', ' ')
print(short_name)

In [ ]:
print(re.search(r"Appendix [A-Z].*", df.html_with_citations.iloc[0], re.S).group())

In [ ]:
## CLEAN OUT HEADERS, APPENDICES AND TABLES

appendix_and_tables = re.search(r"Appendix [A-Z] to(.)+?(Table \d)", df.html_with_citations.iloc[0], re.S).group()

re_appx = re.compile(r"Appendix [A-Z] to ,opinion of.*|APPENDIXES TO OPINION.*", re.S)
#us_const = r"U(U\. S\.) Const\.,? (((a|A)rt\.?|(a|A)mend\.?|(p|P)mbl\.?|(p|P)reamble)( ?[XVI]+))?((, (s|S|&sect;|&#167) ([0-9]+)) ?(, cl\. ([0-9]+)\.?)?)"
amendments = re.compile(r"((first|second|third|fourth|fifth|sixth|seventh|eighth|ninth|tenth|eleventh|twelfth|thirteenth|fourteenth|fifteenth|sixteenth|seventeenth|eighteenth|nineteenth|twentienth|twenty(-)?first|twenty(-)?second|twenty(-)?third|twenty(-)?fourth|twenty(-)?fifth|twenty(-)?sixth|twenty(-)?seventh))+( amendment)( ?[XVI]+)", re.I)


def compile_regexp():
    f_supp = r"([0-9]+ (F\. Supp\. 2d\.|F\. ?Supp\.) [0-9]+)"
#    us_const = r"(U\. S\.) Const\.,? (((a|A)rt\.?|(a|A)mend\.?|(p|P)mbl\.?|(p|P)reamble)( ?[XVI]+))?((, (s|S|&sect;|&#167) ([0-9]+)) ?(, cl\. ([0-9]+)\.?)?)"
def remove_appendices(string):
    regex = re.compile(r"Appendix [A-Z] to ,opinion of.*|APPENDIXES TO OPINION.*", re.S)
    replacement = ''
    string = re.sub(regex, '', string)
    return string
def remove_syllabus_and_headers(string):
        string = re.sub(r'.+?(?=Opinion of)', '', string, count=1) # removes syllabus
        string = re.sub(r"/\.x", "", string)
        string = re.sub(r'NOTICE.*?to press\.', '', string, count=1) # removes slip op notice
        string = re.sub(r"SUPREME COURT OF THE UNITED STATES[_\s]+No\. \d+–\d+[_\s]+", "", string) # remove header
        string = re.sub(r"(\b\d+)?([\s]*)?((Cite as:)? \d{2,3} U\. S\. (\d|_){4} \(\d{4}\))[\s]+\d+[\s]+(Opinion of [A-Z]+, (C\.)? J\.)?(, )?(concurring|dissenting)?(in part)?(and)?(concurring|dissenting)?(in part)?(in judgment)?[\s]+?", "", string) # partial header
        string = re.sub(r"(\b\d+)?([\s]*)?((Cite as:)? \d{2,3} U\. S\. (\d|_){4} \(\d{4}\))[\s]+\d+[\s]+(Opinion of [A-Z]+, (C\.)? J\.)?[\s]+?[A-Z\.,\s]+(, )?(concurring|dissenting)?(in part)?(and)?(concurring|dissenting)?(in part)?[\s]+?(in judgment)?", "", string)
        string = re.sub(r"\b[\d\s]+([A-Z'\s]+ v\. [A-Z'’\s]+)[\s]+(Syllabus)?[\s]+", "", string) # inline citations
        #string = re.sub(r"\s+Cite as: ?\d+ U\. ?S\. ?[_\d]+ \(\d{4}\)?\s+[\d]+?\s+?", '', string, re.S) # removes citation headers
        #string = re.sub(r"[\d]\s.*?Opinion of.*?, (C\. )?J\.", '', string, re.S) # Removes inline citations
        #string = re.sub(r"(?<=SUPREME COURT OF THE UNITED STATES)(.*?, APPELLANT(S)?) v\. .*?(?=APPEAL FROM)", '', string)
        #string = re.sub(r"SUPREME COURT OF THE UNITED STATES.*?\d{4}\]", '', string, re.S)
        return string
        #[\s]+?[A-Z\.,\s]+

# Data cleaning and more data cleaning

* BeautifulSoup
* Citations
* Anomalies

In [ ]:
def court_soup(opinion):
    soup = BeautifulSoup(opinion)
    opinion = soup.text
    opinion = re.sub(r'(\n)+', '', opinion)
    opinion = re.sub(r'(\n)', ' ', opinion)
    return opinion

Remove HTML tags from HTML with citations column

In [ ]:
soup = BeautifulSoup(df.iloc[64029].html_with_citations)
soup.text

In [ ]:
df['clean_text'] = df['html_with_citations'].apply(lambda x: court_soup(x))

Drop columns that aren't helping (because they're null, have low/no variance, are not germane to this project, or are available elsewhere)

In [ ]:
to_drop = ['date_created','author_str','date_created',
           'date_modified','download_url','extracted_by_ocr','html_columbia',
           'html_lawbox','local_path','opinions_cited','joined_by','resource_uri','sha1','type']

In [ ]:
df.drop(to_drop, axis=1, inplace=True)

In [ ]:
# Pickle df
# import pickle
# with open("scotus_df.pickle", "wb") as f:
#     pickle.dump(df, f)
# f.close()

## Get citation data

In [ ]:
cites = glob.glob('data/scotus_clusters/*.json')

In [ ]:
citations = []
for cite in cites:
    with open(cite, 'r') as f:
        data = json.loads(f.read(), object_hook=EmptyDict)
        citations.append(data)

In [ ]:
citedf = pd.DataFrame(citations)
# pickle df
# with open("cites_df.pickle", "wb") as d:
#    pickle.dump(citedf, d)
# d.close()

In [ ]:
citedf.columns

In [ ]:
df.columns

In [ ]:
df.cluster.iloc[0].str.replace()

In [ ]:
citedf.id.iloc[0]

## !!! For resuming work

In [12]:
with open("scotus_df.pickle", "rb") as f:
    sc = pickle.load(f)
with open("cites_df.pickle", "rb") as c:
    cd = pickle.load(c)

# How many cases are there per amendment of interest?

In [ ]:
first = df['clean_text'].str.findall(r"(first amend(\.|ment)?|1st amend(\.|ment)?|(U(\.)? ?S(\.)?)? const(.)? ?amend(\.|ment)? I\b|amend(\.|ment) I\b)", re.I)
    #r"(first amend(\.|ment)|1st amend(\.|ment)|U(\.)? ?S(\.)? const(.)?amend(\.|ment) I)", re.I|re.S)

In [ ]:
# 1444
first = first.apply(lambda x: len(x) > 0)
has_first = df[first].copy()
has_first

In [ ]:
# 1839
fifth = df['clean_text'].str.findall(r"(fifth amend(\.|ment)?|5th amend(\.|ment)?|(U(\.)? ?S(\.)?)? const(.)? ?amend(\.|ment)? V\b|amend(\.|ment) V\b)", re.I|re.S)
fifth = fifth.apply(lambda x: len(x) > 0)
has_fifth = df[fifth].copy()
has_fifth

In [ ]:
# 855
sixth = df['clean_text'].str.findall(r"(sixth amend(\.|ment)?|6th amend(\.|ment)?|(U(\.)? ?S(\.)?)? const(.)? ?amend(\.|ment)? VI\b|amend(\.|ment) VI\b)", re.I|re.S)
sixth = sixth.apply(lambda x: len(x) > 0)
has_sixth = df[sixth].copy()
has_sixth

In [ ]:
# 949
fourth = df['clean_text'].str.findall(r"(fourth amend(\.|ment)?|4th amend(\.|ment)?|(U(\.)? ?S(\.)?)? const(.)? ?amend(\.|ment)? IV\b|amend(\.|ment) IV\b)", re.I|re.S)
fourth = fourth.apply(lambda x: len(x) > 0)
has_fourth = df[fourth].copy()
has_fourth

In [ ]:
# 163
second = df['clean_text'].str.findall(r"(\bsecond amend(\.|ment)?|2nd amend(\.|ment)?|(U(\.)? ?S(\.)?)? const(.)? ?amend(\.|ment)? II\b|amend(\.|ment) II\b)", re.I|re.S)
second = second.apply(lambda x: len(x) > 0)
has_second = df[second].copy()
has_second

In [26]:
# 4579
fourteenth = df['clean_text'].str.findall(r"(fourteenth amend(\.|ment)?|14th amend(\.|ment)?|(U(\.)? ?S(\.)?)? const(.)? ?amend(\.|ment)? XIV\b|amend(\.|ment) XIV\b)", re.I|re.S)
fourteenth = fourteenth.apply(lambda x: len(x) > 0)
has_fourteenth = df[fourteenth].copy()
has_fourteenth

## Prepare dataframe for processing

#### Look for the target amendment (starting with: 14th Amendment)

In [ ]:
# 4579 cases 
has_fourteenth

#### Create key to merge citation information with cases

In [27]:
cluster_keys = has_fourteenth.cluster.str.extractall(r'(?<=\/)(\d+)(?=\/)')
cluster_keys = cluster_keys.unstack()
cluster_keys.columns = ['cluster_key']

has_fourteenth['cluster_key'] = cluster_keys

In [ ]:
cluster_keys

In [28]:
# citedf.id
has_fourteenth['cluster_key'] = has_fourteenth['cluster_key'].apply(lambda x: int(x))

In [29]:
# Merge citations with cases
cases_df = pd.merge(has_fourteenth, citedf,
    how='left',
    left_on='cluster_key',
    right_on='id')

In [30]:
cases_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4579 entries, 0 to 4578
Data columns (total 55 columns):
absolute_url_x               4579 non-null object
author                       3322 non-null object
cluster                      4579 non-null object
html                         4528 non-null object
html_with_citations          4579 non-null object
id_x                         4579 non-null int64
page_count                   215 non-null float64
per_curiam                   4579 non-null bool
plain_text                   4579 non-null object
clean_text                   4579 non-null object
cluster_key                  4579 non-null int64
absolute_url_y               4578 non-null object
attorneys                    4578 non-null object
blocked                      4578 non-null object
case_name                    4578 non-null object
case_name_full               4578 non-null object
case_name_short              4578 non-null object
citation_count               4578 non-null floa

In [ ]:
# cases_df.columns
# cases_df.info()

In [31]:
to_drop_2 = ['absolute_url_x',
'cluster',
'html_with_citations',
'html',
'plain_text',
'cluster_key',
'absolute_url_y',
'attorneys',
'blocked',
'case_name_full',
'case_name_short',             
'date_blocked',
'date_created',
'date_filed_is_approximate',
'date_modified',
'federal_cite_one',
'federal_cite_three',
'federal_cite_two', 
'id_y',
'neutral_cite',     
'scotus_early_cite',
'slug',
'specialty_cite_one',                                                            
'state_cite_one',                                                                
'state_cite_regional',                                                          
'state_cite_three',                                                              
'state_cite_two',
'page_count',
'procedural_history',
'syllabus',
'westlaw_cite']      

In [32]:
# Drop columns
cases_df.drop(to_drop_2, axis=1, inplace=True)

In [ ]:
# Are they really duplicates? Masterpiece isn't really...
dupes = cases_df[cases_df.duplicated(subset='case_name')].copy()

In [ ]:
dupes.iloc[0]['clean_text']
dupes.loc[4383]['clean_text']

### Make sure to extract the year.

In [ ]:
cases_df["year"] = cases_df["date_filed"].apply(lambda x: int(x[:4]))

In [33]:
# pickle
with open("target_df_2.pickle", "wb") as t:
    pickle.dump(cases_df, t)
t.close()

In [38]:
# cases_df.to_csv("target_df_2.csv", index=False)

## Preprocessing

In [51]:
import string

In [ ]:
import gensim
from gensim import corpora
from gensim.models import LsiModel
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
   

In [72]:
import nltk
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk import pos_tag
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer # or LancasterStemmer, RegexpStemmer, SnowballStemmer
from nltk.stem.wordnet import wordnet, WordNetLemmatizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

In [ ]:
# with open("target_df", "rb") as readfile:
#   rcases_df = pickle.load(readfile)

In [ ]:
rcases_df.isna().any()

In [ ]:
# rcases_df.to_csv("target_df.csv")

# !!!!!!!

### Tokenize, parse, POS tag, lemmatize, NER, stop words

In [40]:
r = pd.read_csv('target_df_2.csv')

In [42]:
r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4579 entries, 0 to 4578
Data columns (total 24 columns):
author                      3322 non-null object
id_x                        4579 non-null int64
per_curiam                  4579 non-null bool
clean_text                  4579 non-null object
case_name                   4578 non-null object
citation_count              4578 non-null float64
citations                   4578 non-null object
date_filed                  4578 non-null object
docket                      4578 non-null object
judges                      3827 non-null object
lexis_cite                  4107 non-null object
nature_of_suit              0 non-null float64
non_participating_judges    4578 non-null object
panel                       4578 non-null object
posture                     0 non-null float64
precedential_status         4578 non-null object
resource_uri                4578 non-null object
scdb_decision_direction     4106 non-null float64
scdb_id         

In [48]:
#import lexnlp.nlp.en.transforms.tokens
import lexnlp
import lexnlp.extract.en.courts
from typing import Generator

import regex as re
from reporters_db import EDITIONS, REPORTERS

__author__ = "ContraxSuite, LLC; LexPredict, LLC"
__copyright__ = "Copyright 2015-2019, ContraxSuite, LLC"
__license__ = "https://github.com/LexPredict/lexpredict-lexnlp/blob/master/LICENSE"
__version__ = "0.2.6"
__maintainer__ = "LexPredict, LLC"
__email__ = "support@contraxsuite.com"

CITATION_PTN = r"""
(?:[\s,:\(]|^)
(
(\d+)\s+
({reporters})\s+
(\d+)
(?:,\s+(\d+(?:\-\d+)?))?
(?:\s+\((.+?)?(\d{{4}})\))?
)
(?:\W|$)
""".format(reporters='|'.join([re.escape(i) for i in EDITIONS]))
CITATION_PTN_RE = re.compile(CITATION_PTN, re.IGNORECASE | re.MULTILINE | re.DOTALL | re.VERBOSE)


def get_citations(text, return_source=False, as_dict=False) -> Generator:
    """
    Get citations.
    :param text:
    :param return_source:
    :param as_dict:
    :return: tuple or dict
    (volume, reporter, reporter_full_name, page, page2, court, year[, source text])
    """
    #https://github.com/freelawproject/reporters-db/blob/master/reporters_db/data/reporters.json
    for source_text, volume, reporter, page, page2, court, year\
            in CITATION_PTN_RE.findall(text):
        try:
            reporter_data = REPORTERS[EDITIONS[reporter]]
            reporter_full_name = ''
            if len(reporter_data) == 1:
                reporter_full_name = reporter_data[0]['name']
            elif year:
                for period_data in reporter_data:
                    if reporter in period_data['editions']:
                        start = period_data['editions'][reporter]['start'].year
                        end = period_data['editions'][reporter]['end']
                        if (end and start <= int(year) <= end.year) or start <= int(year):
                            reporter_full_name = period_data['name']
            item = (int(volume),
                    reporter,
                    reporter_full_name,
                    int(page),
                    page2 or None,
                    court.strip(', ') or None,
                    int(year) if year.isdigit() else None)
            if return_source:
                item += (source_text.strip(),)
            if as_dict:
                keys = ['volume', 'reporter', 'reporter_full_name',
                        'page', 'page2', 'court', 'year', 'citation_str']
                item = {keys[n]: val for n, val in enumerate(item)}
            yield item
        except KeyError:
            pass

In [63]:
def stop_citation_noise(text):
    stops = []
    for cite in get_citations(text, return_source=True, as_dict=True):
        # if len(cite['citation_str']) < 70:
            # text = text.replace(cite["citation_str"], " ")
            #    cite['citation_str'] = cite['citation_str'].replace(str(cite['volume']) + cite['reporter'] + str(cite['page']), " ")
            # stops.append(cite["citation_str"])        
        buildcite = str(cite['volume']) + " " + cite['reporter'] + " " + str(cite['page'])
        if cite['page2']:
            buildcite += ', '+str(cite['page2'])
        stops.append(buildcite)
        
    return stops
        
def replace_citations(text):
    stops = stop_citation_noise(text)
    for stop in stops:
        text = text.replace(stop, " ")
    return text

In [90]:
from nltk import pos_tag
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer # or LancasterStemmer, RegexpStemmer, SnowballStemmer
from nltk.stem.wordnet import wordnet, WordNetLemmatizer

default_lemmatizer = WordNetLemmatizer()
default_stemmer = PorterStemmer()
default_stopwords = set(stopwords.words('english')) 

def get_wordnet_pos(word):
# Map POS tag to first character lemmatize() accepts
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN) # it's a noun if it's not found

def tokenize_text(text):
    return [w for s in sent_tokenize(text) for w in word_tokenize(s)]

def remove_special_characters(text, characters=string.punctuation.replace('-', '')):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(characters)))
    return ' '.join(filter(None, [pattern.sub('', t) for t in tokens]))

def lemmatize_text(text, lemmatizer=default_lemmatizer):
    tokens = tokenize_text(text)
    return ' '.join([lemmatizer.lemmatize(t, get_wordnet_pos(t)) for t in tokens])

def stem_text(text, stemmer=default_stemmer):
    tokens = tokenize_text(text)
    return ' '.join([stemmer.stem(t) for t in tokens])

def remove_stopwords(text, stop_words=default_stopwords):
    tokens = [w for w in tokenize_text(text) if w not in stop_words]
    return ' '.join(tokens)
    
    # cleaning pipeline in this function: 
    # remove extra spaces, lowercase, remove stopwords, stem_or_lem
    
def clean_text(text, stem_or_lem = 'lem'):
    text = replace_citations(text)
    #text = text.strip(' ') # strip whitespaces
    text = re.sub(r"[\d]+", " ", text)
    text = text.lower() # lowercase
    text = remove_special_characters(text) # remove punctuation and symbols
    text = remove_stopwords(text) # remove stopwords
    if stem_or_lem == 'stem':
        text = stem_text(text) # stemming
    elif stem_or_lem == 'lem':
        text = lemmatize_text(text) # lemmatizing
    else: # intentionally breaking the argument so neither occurs
        pass 

    return text

In [97]:
r['corpora'] = r['clean_text'].apply(lambda x: clean_text(x))

In [108]:
r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4579 entries, 0 to 4578
Data columns (total 25 columns):
author                      3322 non-null object
id_x                        4579 non-null int64
per_curiam                  4579 non-null bool
clean_text                  4579 non-null object
case_name                   4578 non-null object
citation_count              4578 non-null float64
citations                   4578 non-null object
date_filed                  4578 non-null object
docket                      4578 non-null object
judges                      3827 non-null object
lexis_cite                  4107 non-null object
nature_of_suit              0 non-null float64
non_participating_judges    4578 non-null object
panel                       4578 non-null object
posture                     0 non-null float64
precedential_status         4578 non-null object
resource_uri                4578 non-null object
scdb_decision_direction     4106 non-null float64
scdb_id         

In [99]:
r.to_csv("cleaned_corpora.csv",index=False)

In [107]:
import dask.dataframe as ddf

dask_dataframe = ddf.from_pandas(df, npartitions=12)

,author,id_x,per_curiam,clean_text,case_name,citation_count,citations,date_filed,docket,judges,...,precedential_status,resource_uri,scdb_decision_direction,scdb_id,scdb_votes_majority,scdb_votes_minority,source,sub_opinions,year,corpora


## Create overlapping year ranges

In [ ]:
first_year = 1780
last_year  = 2018
increment  = 20
overlap    = increment // 2
# ------------------------

# ------------------------

# intended use: include left, exclude right
# half-closed, half-open interval [a, b)
def build_year_ranges(first, last, inc, over):
    year_ranges = []
    for n in range(first, last, over):
        year_ranges.append((n, n + inc))
    return year_ranges


# warning: years must have the same index as data
def put_data_under_year_ranges(data, years, year_ranges):

    # assert len(data) == len(years), \
    # "get_content_under_ranges: data and years do not match length"

    # build a dict with keys = year_ranges, with a list for each range
    data_ranges = dict()
    for y in year_ranges:
        data_ranges[y] = []

    # bin all the data by range - each row should fall in two bins, 
    # if ranges are cleanly overlapped

    # if data is a list
    for i in range(len(data)):
        for y in year_ranges:
            if y[0] <= years[i] and years[i] < y[1]:
                data_ranges[y].append(data[i])
                # this should happen twice for every entry except 
                # the very oldest and the very newest

    # pandas df

    return data_ranges

# ------------

def run_year_range_build(): # main
    
    cases = []
    years = []
    corpora = []
                  
    for i in range(500):
        years.append(first_year, last_year)
        i = 0, len(corpora)-1
        j = 0, len(corpora)-1
        cases.append(corpora[i] + ' ' + corpora[j])
        
    # and bin them
    bins = build_year_ranges(first_year, last_year, increment, overlap)
    binned_data = put_data_under_year_ranges(cases, years, bins)

In [ ]:
with open("corpus.pickle", "wb") as f:
    pickle.dump(corpus, f)

## Create ngrams

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def wm_to_df(wm, feat_names):
    # create an index for each row
    doc_names = ['Doc{:d}'.format(idx) for idx, _ in enumerate(wm)]
    df = pd.DataFrame(data=wm.toarray(), index=doc_names,
                      columns=feat_names)
    return(df)

def make_ngrams(docs):
    custom_vec = CountVectorizer(ngram_range=(2,3))
    corpora = []
    for doc in tqdm.tqdm(nlp.pipe(docs)):
        doc = [token.lemma_ for token in doc if (not token.is_stop and not token.is_digit and not token.is_punct)]
        doc = " ".join(doc)
        corpora.append(doc)
    wm = custom_vec.fit_transform(corpora)
    tokens = custom_vec.get_feature_names()
    df = wm_to_df(wm, tokens)
    return df

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[100]]])

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

## possible LSA workflow for sklearn

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline

In [ ]:
# documents = 

In [ ]:
# vectorizer to convert raw documents to TF/IDF matrix

vectorizer = TfidfVectorizer(stop_words='english', 
                             use_idf=True, 
                             smooth_idf=True)

# Normalizes the vector (L2 norm of 1.0) to normalize 
# the effect of document length on tf-idf

normalizer = Normalizer(copy=False)

In [ ]:
# Perform SVD
# Project the tfidf vectors onto the first N principal components.

svd_model = TruncatedSVD(n_components=100,         // num dimensions
                         algorithm='randomized',
                         n_iter=10)

lsa_transformer = Pipeline([('tfidf', vectorizer), 
                            ('svd', svd_model),
                            ('norm', normalizer)])


In [ ]:
lsa_matrix = lsa_transformer.fit_transform(documents)

print("Number of tf-idf features: {svd_matrix.get_shape()[1]}")

# Get the words that correspond to each of the features.
feat_names = vectorizer.get_feature_names()

In [ ]:
#for component_num in range(0, 100, 10):
for component_num in range(0, 10):

    comp = svd.components_[component_num]
    
    # Sort the weights in the first component and get indices
    indices = numpy.argsort(comp).tolist()
    
    # Reverse order (largest weights first)
    indices.reverse()
    
    # Get top 10 terms for component        
    terms = [feat_names[weight_index] for weight_index in indices[0:10]]    
    weights = [comp[weight_index] for weight_index in indices[0:10]]    
   
    # Display these terms and their weights as a horizontal bar graph.    
    # The horizontal bar graph displays the first item on the bottom; reverse
    # the order of the terms so the biggest one is on top.
    terms.reverse()
    weights.reverse()
    positions = arange(10) + .5    # the bar centers on the y axis
    
    figure(component_num)
    barh(positions, weights, align="center")
    yticks(positions, terms)
    xlabel("Weight")
    title("Strongest terms for component {component_num}")
    grid(True)
    show()

In [ ]:
## Possible PLSA workflow

In [ ]:
## Necessary visualization